In [5]:
%matplotlib inline

from dask.distributed import Client
import dask
from map.generate_map_weights import *
from datetime import datetime, timedelta
import grids.config as config
from grids.utils import get_cache_dir, make_parent_dir, profile

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.35 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46853,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.35 GiB
Comm: tcp://127.0.0.1:45919,Total threads: 2
Dashboard: http://127.0.0.1:41773/status,Memory: 3.84 GiB
Nanny: tcp://127.0.0.1:33935,


In [8]:
# Setup some criteria
ingest_days = 1
forecast_interval_hrs = 6
start_dt = datetime(2023, 1, 1, 18) # First one is at 00Z in date
td = timedelta(hours=forecast_interval_hrs)
number_of_forecasts = 1 #int(ingest_days * 24/forecast_interval_hrs)

In [10]:
# Loop though forecasts, fetch and insert
for f in range(number_of_forecasts):
    reference_time = start_dt + td * f
    ref_time_str = reference_time.strftime("%Y%m%dT%HZ")

    print(f"Start download of {ref_time_str}")

    blob_list = list_blobs_forcing(
        configuration = "forcing_medium_range",
        reference_time = ref_time_str,
        must_contain = "forcing"
    )

    # This can be used to run serial
    dfs = []
    for blob_name in blob_list:
        df = dask.delayed(mp.calculate_map_forcing)(blob_name, use_cache=True)
        dfs.append(df)
    
    # Join all timesteps into single pd.DataFrame
    %time 
    results = dask.compute(*dfs)
    df = pd.concat(results)

    # Save as parquet file
    parquet_filepath = os.path.join(config.MEDIUM_RANGE_PARQUET, f"{ref_time_str}.parquet")
    make_parent_dir(parquet_filepath)
    df.to_parquet(parquet_filepath)

    # Print out some DataFrame stats
    print(df.info(verbose=True, memory_usage='deep'))
    print(df.memory_usage(index=True, deep=True))


Start download of 20230101T18Z
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.34 µs


2023-01-16 17:30:02,215 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2023-01-16 17:30:02,398 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2023-01-16 17:30:02,667 - distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
2023-01-16 17:30:02,862 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2023-01-16 17:30:04,680 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2023-01-16 17:30:04,817 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-01-16 17:30:04,872 - distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
2023-01-16 17:30:05,019 - distributed.utils_perf - WARNING - full garbage collections took

2023-01-16 17:30:16,408 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.92 GiB -- Worker memory limit: 3.84 GiB
2023-01-16 17:30:16,506 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.92 GiB -- Worker memory limit: 3.84 GiB
2023-01-16 17:30:16,606 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

2023-01-16 17:30:30,098 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2023-01-16 17:30:30,306 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2023-01-16 17:30:30,537 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2023-01-16 17:30:30,768 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2023-01-16 17:30:32,143 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2023-01-16 17:30:32,155 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2023-01-16 17:30:32,362 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2023-01-16 17:30:32,377 - distributed.utils_perf - WARNING - full garbage collections took

2023-01-16 17:30:39,588 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2023-01-16 17:30:39,670 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.73 GiB -- Worker memory limit: 3.84 GiB
2023-01-16 17:30:39,770 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.75 GiB -- Worker memory limit: 3.84 GiB
2023-01-16 17:30:39,869 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to t

...

2023-01-16 17:32:58,888 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.87 GiB -- Worker memory limit: 3.84 GiB
2023-01-16 17:32:58,989 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.87 GiB -- Worker memory limit: 3.84 GiB
2023-01-16 17:32:59,301 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

2023-01-16 17:33:05,387 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.94 GiB -- Worker memory limit: 3.84 GiB
2023-01-16 17:33:05,488 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.94 GiB -- Worker memory limit: 3.84 GiB
2023-01-16 17:33:05,802 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

2023-01-16 17:33:31,019 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2023-01-16 17:33:31,417 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
2023-01-16 17:33:31,689 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2023-01-16 17:33:32,331 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-01-16 17:33:32,600 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-01-16 17:33:34,198 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2023-01-16 17:33:34,502 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2023-01-16 17:33:34,763 - distributed.utils_perf - WARNING - full garbage collections took

2023-01-16 17:33:44,987 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.83 GiB -- Worker memory limit: 3.84 GiB
2023-01-16 17:33:45,087 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 3.03 GiB -- Worker memory limit: 3.84 GiB
2023-01-16 17:33:45,188 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3879360 entries, 0 to 16163
Data columns (total 7 columns):
 #   Column            Dtype         
---  ------            -----         
 0   catchment_id      category      
 1   value             float32       
 2   reference_time    datetime64[ns]
 3   value_time        datetime64[ns]
 4   configuration     category      
 5   measurement_unit  category      
 6   variable_name     category      
dtypes: category(4), datetime64[ns](2), float32(1)
memory usage: 123.6 MB
None
Index               31034880
catchment_id         9370132
value               15517440
reference_time      31034880
value_time          31034880
configuration        3879545
measurement_unit     3879532
variable_name        3879543
dtype: int64
